In [20]:
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import isnan

In [21]:
#load features from all images
feats = np.load('MobileNet_Features.npy')

In [22]:
#load meta information
files = pd.read_csv("metadata.csv")

In [23]:
# Get opacity score and paitent ID
PID = np.array(files.patientid)
Score = np.array(files.score)

In [24]:
#Extract Feature from each paitent with last two feature difference method and assign corresponding radiological trajectory labels
LastID = PID[0]
LastScore = Score[0]
Currentfeat = feats[0]
Label = []
allfeats = []
allIDS = []
allContinent = []
for i in range(1,259):
    NewID = PID[i]
    NewScore = Score[i]
    if (NewID == LastID):
        Currentfeat = feats[i]-Currentfeat
        allfeats.append(Currentfeat)
        Score_dif = NewScore-LastScore
        if Score_dif>0.3:
            Label.append(1)
        elif Score_dif<-0.3:
            Label.append(-1)
        else:
            Label.append(0)
        allIDS.append(PID[i])
    else:
        LastID = NewID
        Currentfeat = feats[i]
    LastScore = NewScore
feats_difference = np.array(allfeats)
np.save('Feature_Difference_MobileNet',feats_difference)

In [25]:
#Extract Feature from each paitent with all features concatenation method and assign corresponding radiological trajectory labels
LastID = PID[0]
LastScore = Score[0]
Currentfeat = feats[0]
Label = []
allfeats = []
allIDS = []


for i in range(1,259):
    NewID = PID[i]
    NewScore = Score[i]
    if (NewID == LastID):
        Currentfeat = np.vstack((Currentfeat,feats[i]))
        allfeats.append(Currentfeat)
        Score_dif = NewScore-LastScore
        if Score_dif>0.3:
            Label.append(1)
        elif Score_dif<-0.3:
            Label.append(-1)
        else:
            Label.append(0)
    else:
        LastID = NewID
        Currentfeat = feats[i]
    LastScore = NewScore

In [26]:
#Zero Padding
pad2 = np.zeros((2,Currentfeat.shape[1]))
pad3 = np.zeros((1,Currentfeat.shape[1]))
allfestspad = []
for i in range(len(allfeats)):
    if(allfeats[i].shape[0]==2):
        allfestspad.append(np.vstack((pad2,allfeats[i])))
    elif(allfeats[i].shape[0]==3):
        allfestspad.append(np.vstack((pad3,allfeats[i])))
    else:
        allfestspad.append(allfeats[i])
feats_concatenation = np.array(allfestspad)
np.save('Feature_Concatenation_MobileNet',feats_concatenation)

In [27]:
#save radiological trajectory labels
labels = np.array(Label)
np.save('Labels',labels)